In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.model_selection import train_test_split

data_path = '/content/drive/MyDrive/dataset/Final-Fashion-Dataset.csv'
df = pd.read_csv(data_path)

In [ ]:
# Preprocessing
categorical_features = ['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']
numerical_features = ['year', 'Price (USD)']
id_features = ['user-id', 'id']

In [ ]:
# 1. Encode categorical features
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])

In [ ]:
# 2. Scale numerical features
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
# 3. Define input layers
categorical_inputs = []
for feature in categorical_features:
    input_layer = Input(shape=(1,), name=feature)
    categorical_inputs.append(input_layer)

numerical_input = Input(shape=(len(numerical_features),), name='numerical_features')


In [ ]:
# 4. Embedding layers for categorical features
embedding_layers = []
for input_layer, feature in zip(categorical_inputs, categorical_features):
    embedding_layer = Embedding(input_dim=df[feature].nunique(), output_dim=10)(input_layer)  # Adjust output_dim as needed
    embedding_layers.append(embedding_layer)

In [ ]:
# 5. Flatten embedding layers
flatten_layers = []
for embedding_layer in embedding_layers:
    flatten_layer = Flatten()(embedding_layer)
    flatten_layers.append(flatten_layer)

# 6. Concatenate all layers
all_features = Concatenate()([numerical_input] + flatten_layers)

In [ ]:
# 7. Define dense layers (DNN architecture)
x = Dense(128, activation='relu')(all_features)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='linear')(x)  # Assuming a regression task (predicting 'Price (USD)')

# 8. Create the model
model = Model(inputs=[numerical_input] + categorical_inputs, outputs=output)

# Compile the model (define optimizer, loss function, metrics)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Prepare data for training
# 1. Separate features and target variable
X = df[['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage', 'year', 'Price (USD)']]
y = df['Price (USD)']  # Assuming 'Price (USD)' is the target variable

# 2. Create input data for the model
X_numerical = X[numerical_features].values
# Reshape X_categorical to be a single array with samples as rows and features as columns
X_categorical = X[categorical_features].values

# 3. Split data into training and testing sets
X_train_num, X_test_num, X_train_cat, X_test_cat, y_train, y_test = train_test_split(
    X_numerical, X_categorical, y, test_size=0.2, random_state=42
)

# Train the model
model.fit(
    x=[X_train_num] + [X_train_cat[:, i] for i in range(X_train_cat.shape[1])],  # Input data - list of arrays, one for each input layer
    y=y_train,  # Target variable
    epochs=10,  # Number of training epochs
    batch_size=32,  # Batch size
    validation_data=([X_test_num] + [X_test_cat[:, i] for i in range(X_test_cat.shape[1])], y_test)  # Validation data - list of arrays, one for each input layer
)

Epoch 1/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0067 - mae: 0.0276 - val_loss: 4.0164e-06 - val_mae: 0.0015
Epoch 2/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 3.7497e-06 - mae: 0.0015 - val_loss: 1.7113e-06 - val_mae: 9.7268e-04
Epoch 3/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.4604e-06 - mae: 0.0012 - val_loss: 1.8266e-06 - val_mae: 0.0011
Epoch 4/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.3113e-06 - mae: 0.0012 - val_loss: 8.9293e-07 - val_mae: 6.8860e-04
Epoch 5/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 3.0101e-06 - mae: 0.0013 - val_loss: 2.7724e-06 - val_mae: 0.0014
Epoch 6/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 6.2313e-06 - mae: 0.0019 - val_loss: 5.0689e-06 - val_mae: 0.0021
Epoch 7/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2.0428e-06 - mae: 0.0011 - val_loss: 3.9514e-04 - val_mae: 0.0197
Epoch 8/10
913/913 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 1.5165e-05 - mae: 0.0019 - val_loss: 3.4388e-06 - val_

In [ ]:
def recommend_items_for_user(user_id, top_k=5):
    """Recommends top_k items for a given user_id.

    Args:
        user_id: The ID of the user.
        top_k: The number of items to recommend.

    Returns:
        A list of item IDs representing the recommendations.
    """

    # 1. Filter items not yet interacted with by the user (if applicable)
    # Assuming you have an interaction history, you can filter items the user has already interacted with.
    # This might require modifications based on your data structure.

    all_items = df['id'].unique()  # Example: Get all unique item IDs
    # ... (filter out items already interacted with) ...

    # 2. Create input data for the model for each candidate item
    # For simplicity, we'll predict the price for all items and then rank them based on the prediction

    input_data_num = []
    input_data_cat = [[] for _ in range(len(categorical_features))]

    for item_id in all_items:
        item_data = df[df['id'] == item_id].iloc[0]  # Get data for the item

        # Extract numerical features
        input_data_num.append(item_data[numerical_features].values.astype('float32'))

        # Extract categorical features
        for i, feature in enumerate(categorical_features):
            input_data_cat[i].append(item_data[feature])

    input_data_num = np.array(input_data_num)
    input_data_cat = [np.array(cat_data) for cat_data in input_data_cat]

    # 3. Predict prices for all items
    predicted_prices = model.predict([input_data_num] + input_data_cat)

    # 4. Rank items by predicted price (or another relevant metric)
    item_scores = predicted_prices.flatten()  # Assuming lower prices are preferred

    # 5. Get the top_k items
    top_item_indices = np.argsort(item_scores)[:top_k]  # Sort in ascending order for price
    recommended_item_ids = all_items[top_item_indices]

    return recommended_item_ids

# Example usage:
user_id = 36062  # Replace with a user ID
recommendations = recommend_items_for_user(user_id)
print(f"Recommendations for user {user_id}: {recommendations}")

1141/1141 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Recommendations for user 36062: [13727 15371  1789 13146 41669]
